In [1]:
import eegPinelineDesign
import numpy as np
import random
import mne
import matplotlib.pyplot as plt
%matplotlib inline
import os
import pandas as pd
import re
import json
import scipy
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA,FastICA
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import cross_val_score
from scipy.fftpack import fft,ifft
import math
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
from scipy.signal import spectrogram,find_peaks_cwt
from mne.preprocessing.ica import ICA
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import label_binarize
import itertools
from nitime import algorithms as alg

c:\anaconda3\lib\site-packages\pandas\__init__.py:7: DeprecationWarning: bad escape \s
  from pandas import hashtable, tslib, lib
c:\anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility
  return f(*args, **kwds)
c:\anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility
  return f(*args, **kwds)
c:\anaconda3\lib\site-packages\ipykernel\pylab\config.py:66: DeprecationWarning: metadata {'config': True} was set from the constructor.  Metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
  inline backend."""
c:\anaconda3\lib\site-packages\ipykernel\pylab\config.py:71: DeprecationWarning: metadata {'config': True} was set from the constructor.  Metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
  'retina', 'jpeg', 'svg', 'pdf'.""")
c:\anaconda3\lib\site-packages\ipykernel\pylab\

In [2]:
eegPinelineDesign.change_file_directory('C:/Users/ning/Downloads/training set')
EDFfiles, Annotationfiles = eegPinelineDesign.split_type_of_files()
EDFfiles

['64chlocs.elp', 'label_extraction.npy', 'label_extraction.pkl', 's5d2_final.edf', 's5d2_final_annotations.txt', 's6n2_final.edf', 's6n2_final_annotations.txt', 'Sleep Stage Scoring Criteria.docx', 'suj10_d1final.edf', 'suj10_d1final_annotations.txt', 'suj10_d2final.edf', 'suj10_d2final_annotations.txt', 'suj13_l2nap_day2 edited.edf', 'suj13_l2nap_day2 edited1.edf', 'suj13_l2nap_day2 edited1_annotations.txt', 'suj13_l2nap_day2 edited_annotations.txt', 'suj13_l2nap_day2 edited_C3.txt', 'suj13_l2nap_day2 edited_C4.txt', 'suj13_l2nap_day2 edited_F3.txt', 'suj13_l2nap_day2 edited_F4.txt', 'suj13_l2nap_day2 edited_O1.txt', 'suj13_l2nap_day2 edited_O2.txt', 'suj5_d1final.edf', 'suj5_d1final_annotations.txt', 'suj6_d1final.edf', 'suj6_d1final_annotations.txt', 'suj8_d1final.edf', 'suj8_d1final_annotations.txt', 'suj8_d2final.edf', 'suj8_d2final_annotations.txt', 'suj9_d1final.edf', 'suj9_d1final_annotations.txt', 'suj9_d2final.edf', 'suj9_d2final_annotations.txt', 'testing-montage-2.mtg', 'Tr

c:\anaconda3\lib\site-packages\IPython\core\formatters.py:92: DeprecationWarning: DisplayFormatter._ipython_display_formatter_default is deprecated: use @default decorator instead.
  def _ipython_display_formatter_default(self):
c:\anaconda3\lib\site-packages\IPython\core\formatters.py:669: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(self):


['s5d2_final.edf',
 's6n2_final.edf',
 'suj10_d1final.edf',
 'suj10_d2final.edf',
 'suj13_l2nap_day2 edited.edf',
 'suj13_l2nap_day2 edited1.edf',
 'suj5_d1final.edf',
 'suj6_d1final.edf',
 'suj8_d1final.edf',
 'suj8_d2final.edf',
 'suj9_d1final.edf',
 'suj9_d2final.edf']

In [3]:
# initialization and preallocation
TrainingLabel=[]
TrainingData = []
totoal_lenght = 0

file_to_read,fileName = eegPinelineDesign.pick_sample_file(EDFfiles,n = 0)

channelList = ['F3','F4','C3','C4','O1','O2','ROC','LOC']
raw = eegPinelineDesign.load_data(file_to_read,channelList,None,100)
annotation_to_read = [x for x in Annotationfiles if fileName in x]
file = pd.read_csv(annotation_to_read[0])
windowsLabel = eegPinelineDesign.annotation_to_labels(Annotationfiles,fileName)

Extracting edf Parameters from s5d2_final.edf...
Setting channel info structure...
Creating Raw.info structure...
Reading 0 ... 3383999  =      0.000 ...  3383.999 secs...
[done]
Ready.
Fitting ICA to data using 8 channels. 
Please be patient, this may take some time
Inferring max_pca_components from picks.
Using all PCA components: 8
    Searching for artifacts...
    found 2 artifacts by EOG 00
    found 1 artifact by skewness
    found 1 artifact by kurtosis
    found 1 artifact by variance
Artifact indices found:
    5, 6, 0, 0, 7
    Removing duplicate indices...
Ready.
Transforming to ICA space (8 components)
Zeroing out 4 ICA components
Inverse transforming to PCA space
Reconstructing sensor space signals from 8 PCA components
Low-pass filtering at 5e+02 Hz
Low-pass filtering at 1e+02 Hz


In [50]:
Fs=1000;data_to_store=[]
for sample, items in enumerate(windowsLabel):
    if items == 'w':
        TrainingLabel.append(0)
                
    elif items == '1':
        TrainingLabel.append(1)
                
    elif items == '2':
        TrainingLabel.append(2)
                
    elif items == '3':
        TrainingLabel.append(3)
    for index,chan_names in enumerate(['F3','F4','C3','C4','O1','O2']):
        if index == 0:
            startPoint = sample*30;endPoint = (sample+1)*30
            start,stop = raw.time_as_index([startPoint,endPoint])
            tempSegment,time = raw[index,start:stop]
            tempdata = tempSegment
        else:
            startPoint = sample* 30;endPoint = (sample+1)*30
            start,stop = raw.time_as_index([startPoint,endPoint])
            tempSegment, time = raw[index,start:stop]
            tempdata += tempSegment
    
    band_data = []
    delta_band = mne.filter.low_pass_filter(tempdata,Fs,4)
    theta_band = mne.filter.band_pass_filter(tempdata,Fs,4,7.5)
    alpha_band = mne.filter.band_pass_filter(tempdata,Fs,7.5,14)
    beta_band  = mne.filter.band_pass_filter(tempdata,Fs,14,40)
    gamma_band = mne.filter.high_pass_filter(tempdata,Fs,40)
    band_data.append([delta_band,theta_band,alpha_band,beta_band,gamma_band])
    
    before_data_to_store=[]
    for freq_band in band_data[0]:
        coef,sigma = alg.AR_est_LD(np.array(freq_band)[0,:],2)
        before_data_to_store.append(np.append(coef,sigma))
    data_to_store.append(list(np.array(before_data_to_store).ravel()))

In [52]:
np.array(data_to_store).shape

(113, 15)